In [1]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame

In [7]:
dax_40 = {
    "adidas" : web.DataReader("ADS.DE", 'yahoo', one_year_ago, today),
    "airbus" : web.DataReader("AIR.DE", 'yahoo', one_year_ago, today),
    "allianz" : web.DataReader("ALV.DE", 'yahoo', one_year_ago, today),
    "basf" : web.DataReader("BAS.DE", 'yahoo', one_year_ago, today),
}

for k in dax_40.keys():
    dax_40[k] = dax_40[k].sort_values(by="Date")
    dax_40[k] = dax_40[k].dropna()
    dax_40[k] = dax_40[k].assign(close_day_before=dax_40[k].Close.shift(1))
    dax_40[k]['returns'] = ((dax_40[k].Close - dax_40[k].close_day_before)/dax_40[k].close_day_before)
dax_40["adidas"]

,High,Low,Open,Close,Volume,Adj Close,close_day_before,returns
Date,,,,,,,,
2021-05-13,286.299988,277.149994,283.000000,284.850006,501372.0,279.641663,NaN,NaN
2021-05-14,290.000000,285.799988,289.000000,288.549988,438464.0,283.273987,284.850006,0.012989
2021-05-17,297.250000,289.200012,289.399994,296.799988,824420.0,291.373138,288.549988,0.028591
2021-05-18,301.700012,292.299988,299.950012,293.799988,576402.0,288.427979,296.799988,-0.010108
2021-05-19,295.350006,289.350006,293.049988,294.549988,629842.0,289.164276,293.799988,0.002553
...,...,...,...,...,...,...,...,...
2022-05-09,180.960007,173.539993,176.660004,173.539993,855795.0,170.366898,188.220001,-0.077994
2022-05-10,178.880005,173.360001,176.240005,173.360001,881941.0,170.190186,173.539993,-0.001037
2022-05-11,183.860001,175.199997,176.300003,181.860001,0.0,178.534775,173.360001,0.049031


In [2]:
"""
S = Current Stock Price
K = Strike price
t = time to maturity
r = risk-free rate
sigma = valatility
"""
def d1(S,K,t,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*t)/(sigma*sqrt(t))
def d2(S,K,t,r,sigma):
    return d1(S,K,t,r,sigma)-sigma*sqrt(t)

In [3]:
# european call
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

# european put
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

In [10]:
from datetime import datetime, date
import numpy as np
import pandas as pd
import pandas_datareader.data as web

#stock = '^GDAXI'
expiry = '2022-12-14'
strike_price = 140

today = datetime(2022, 5, 18)
#print(today)
one_year_ago = today.replace(year=today.year-1)

company = "adidas"

df = dax_40[company]

sigma = np.sqrt(252) * df['returns'].std()
uty = (web.DataReader(
    "^TNX", 'yahoo', today.replace(day=today.day-1), today)['Close'].iloc[-1])/100
lcp = df['Close'].iloc[-1]
t = (datetime.strptime(expiry, "%Y-%m-%d") - datetime.utcnow()).days / 365


print("+++ " + str(company) + " +++")
print('The Put Option Price is: ', bs_put(lcp, strike_price, t, uty, sigma))
print('The Call Option Price is: ', bs_call(lcp, strike_price, t, uty, sigma))

+++ adidas +++
The Put Option Price is:  2.955657664993538
The Call Option Price is:  45.79482854983834


In [35]:
t = (datetime.strptime(expiry, "%Y-%m-%d") - datetime.utcnow()).days / 365
print(t)

0.34794520547945207
